conv2d的一些基本使用。
使用一个5*5的kernel对一个随机图像做卷积

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc

## 生成卷积核并对随机图片进行卷积操作：单通道不用rgb

In [50]:
k = np.float32([1, 4, 6, 4, 1])
k2 = np.inner(k,k)
print('inner:',k2)#inner equals to 1+16+36+16+1=70
k = np.outer(k, k)
#print('kernel:\n',k)#
k5x5 = k[:,:,None,None] / k.sum() 
print('k5x5 shape:\n',k5x5.shape)
#print('k5x5:\n',k5x5)


img = np.float32(np.random.randint(1,255,8*8))
img = img.reshape(1,8,8,1)
#print(img)

img = tf.nn.conv2d(img, k5x5, [1,1,1,1], 'SAME')

#print("after conv:\n",tf.Session().run(img))

inner: 70.0
k5x5 shape:
 (5, 5, 1, 1)


##  生成卷积核并对随机图片进行卷积操作：rgb扩展，三通道

In [2]:
k = np.float32([1,4,6,4,1])
k = np.outer(k,k)#外积生成矩阵
print('kernel:\n',k)

kernel:
 [[  1.   4.   6.   4.   1.]
 [  4.  16.  24.  16.   4.]
 [  6.  24.  36.  24.   6.]
 [  4.  16.  24.  16.   4.]
 [  1.   4.   6.   4.   1.]]


In [32]:
#三通道扩充
eye = np.eye(3, dtype = np.float32)
k5x5 = k[:,:,None,None] / k.sum() * np.eye(3, dtype = np.float32)#利用前边生成的矩阵扩充到5*5*3*3
print('k5x5 shape:',k5x5.shape)
#print('k5x5:\n',k5x5)

k5x5 shape: (5, 5, 3, 3)


In [30]:
#分解
pre_multiply_k5x5 = k[:,:,None,None]/k.sum()
print(pre_multiply_k5x5.shape)#5*5*1*1
#print(pre_multiply_k5x5)
#乘以eye就是把同一份数据分给三通道。
print(eye)
#归一化：虽然是k的sum，但是刚好等于256,又像素上限是255.
print(k.sum())

(5, 5, 1, 1)
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
256.0


## 利用生成的kernel给一个随机生成的图片做卷积。打印前后两张图片进行对比：经过卷积后，随机嘈杂的原图变的圆滑。

In [38]:
img = np.float32(np.random.randint(1,255,8*8*3))
img = img.reshape(1,8,8,3)
print(img[0].shape)
scipy.misc.toimage(img[0]).save('img_before_conv2d.jpg')
#print(img)
img = tf.nn.conv2d(img,k5x5,[1,2,2,1],'SAME')
#print('after conv2d:',img)

(8, 8, 3)
after conv2d: [[[[  90.0859375   114.90234375  110.19921875]
   [ 117.42578125  122.71484375  123.42578125]
   [  99.3984375   107.375       137.82421875]
   [  68.15234375   67.0390625    99.53515625]]

  [[ 102.6796875   131.76953125  154.16796875]
   [ 145.83203125  111.3828125   127.1171875 ]
   [ 140.4296875   117.03125     119.6640625 ]
   [  66.609375     96.8828125    85.61328125]]

  [[ 119.41796875  146.08984375   98.60546875]
   [ 143.15234375  115.171875    123.12890625]
   [ 116.29296875  121.28515625  131.6875    ]
   [  85.8515625    93.578125     94.66015625]]

  [[  71.6015625    83.70703125   64.23828125]
   [  84.44140625   67.6640625    50.51171875]
   [  79.5625      109.1328125    81.0625    ]
   [  57.77734375   68.6484375    82.1484375 ]]]]


In [44]:
img_conv2d = tf.Session().run(img)
print('after conv2d:',img_conv2d)
print(img_conv2d.shape)
print(img_conv2d[0].shape)
scipy.misc.toimage(img_conv2d[0]).save('img_after_conv2d.jpg')
#scipy.misc.toimage(img[0]).save('before_after_conv2d.jpg')

after conv2d: [[[[  90.0859375   114.90234375  110.19921875]
   [ 117.42578125  122.71484375  123.42578125]
   [  99.3984375   107.375       137.82421875]
   [  68.15234375   67.0390625    99.53515625]]

  [[ 102.6796875   131.76953125  154.16796875]
   [ 145.83203125  111.3828125   127.1171875 ]
   [ 140.4296875   117.03125     119.6640625 ]
   [  66.609375     96.8828125    85.61328125]]

  [[ 119.41796875  146.08984375   98.60546875]
   [ 143.15234375  115.171875    123.12890625]
   [ 116.29296875  121.28515625  131.6875    ]
   [  85.8515625    93.578125     94.66015625]]

  [[  71.6015625    83.70703125   64.23828125]
   [  84.44140625   67.6640625    50.51171875]
   [  79.5625      109.1328125    81.0625    ]
   [  57.77734375   68.6484375    82.1484375 ]]]]
(1, 4, 4, 3)
(4, 4, 3)


In [ ]:
# tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, name=None)
# 除去name参数用以指定该操作的name，与方法有关的一共五个参数：
#
# 第一个参数input：指需要做卷积的输入图像，它要求是一个Tensor，具有[batch, in_height, in_width, in_channels]这样的shape，具体含义是[训练时一个batch的图片数量, 图片高度, 图片宽度, 图像通道数]，注意这是一个4维的Tensor，要求类型为float32和float64其中之一
#
# 第二个参数filter：相当于CNN中的卷积核，它要求是一个Tensor，具有[filter_height, filter_width, in_channels, out_channels]这样的shape，具体含义是[卷积核的高度，卷积核的宽度，图像通道数，卷积核个数]，要求类型与参数input相同，有一个地方需要注意，第三维in_channels，就是参数input的第四维
#
# 第三个参数strides：卷积时在图像每一维的步长，这是一个一维的向量，长度4
#
# 第四个参数padding：string类型的量，只能是"SAME","VALID"其中之一，这个值决定了不同的卷积方式（后面会介绍）
#
# 第五个参数：use_cudnn_on_gpu:bool类型，是否使用cudnn加速，默认为true
#
# 结果返回一个Tensor，这个输出，就是我们常说的feature map

# 一堆卷积例子

## 这个列表结构原本是为了把各个例子都打印出来,现在只做一份，替换参数部分。参数和kernel都做了变形打印，以图形的视角维度为主。

In [17]:

oplist=[]

#######################这部分用不同例子替换##############################
# input:[batch, in_height, in_width, in_channels]
# filter:[filter_height, filter_width, in_channels, out_channels]
input_arg  = tf.Variable(tf.ones([1, 5, 5, 5]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 5 ,1]))
##################################################################
input_display = tf.Variable(tf.ones(input_arg.shape))#只是为了打印
input_display = tf.transpose(input_display,[0,3,1,2])
filter_display = tf.Variable(tf.ones(filter_arg.shape))#只是为了打印
filter_display = tf.transpose(filter_display,[2,3,0,1])
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,1,1,1], use_cudnn_on_gpu=False, padding='VALID')
oplist.append([op2, "case N"])


with tf.Session() as a_sess:
    a_sess.run(tf.global_variables_initializer())
    for aop in oplist:
        print("----------{}---------".format(aop[1]))
        print(a_sess.run(input_display))
        print('filter:\n',a_sess.run(filter_display))
        print(np.transpose(a_sess.run(aop[0]),[0,3,1,2]))
        print('---------------------\n\n')

----------case N---------
[[[[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]

  [[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]

  [[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]

  [[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]

  [[ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]
   [ 1.  1.  1.  1.  1.]]]]
filter:
 [[[[ 1.  1.  1.]
   [ 1.  1.  1.]
   [ 1.  1.  1.]]]


 [[[ 1.  1.  1.]
   [ 1.  1.  1.]
   [ 1.  1.  1.]]]


 [[[ 1.  1.  1.]
   [ 1.  1.  1.]
   [ 1.  1.  1.]]]


 [[[ 1.  1.  1.]
   [ 1.  1.  1.]
   [ 1.  1.  1.]]]


 [[[ 1.  1.  1.]
   [ 1.  1.  1.]
   [ 1.  1.  1.]]]]
[[[[ 45.  45.  45.]
   [ 45.  45.  45.]
   [ 45.  45.  

In [15]:
####################各例，可以替换到上方打印###########################################
input_arg  = tf.Variable(tf.ones([1, 3, 3, 5]))
filter_arg = tf.Variable(tf.ones([1 ,1 , 5 ,1]))
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,1,1,1], use_cudnn_on_gpu=False, padding='VALID')
oplist.append([op2, "case 2"])

#############################################################

input_arg  = tf.Variable(tf.ones([1, 3, 3, 5]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 5 ,1]))
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,1,1,1], use_cudnn_on_gpu=False, padding='VALID')
oplist.append([op2, "case 3"])

#############################################################
input_arg  = tf.Variable(tf.ones([1, 5, 5, 5]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 5 ,1]))

op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,1,1,1], use_cudnn_on_gpu=False, padding='VALID')
oplist.append([op2, "case 4"])

#############################################################
input_arg  = tf.Variable(tf.ones([1, 5, 5, 5]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 5 ,1]))
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,1,1,1], use_cudnn_on_gpu=False, padding='SAME')
oplist.append([op2, "case 5"])

#############################################################
input_arg  = tf.Variable(tf.ones([1, 5, 5, 5]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 5 ,7]))
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,1,1,1], use_cudnn_on_gpu=False, padding='SAME')
oplist.append([op2, "case 6"])

#############################################################

input_arg  = tf.Variable(tf.ones([1, 5, 5, 5]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 5 ,7]))
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,2,2,1], use_cudnn_on_gpu=False, padding='SAME')
oplist.append([op2, "case 7"])

#############################################################

input_arg  = tf.Variable(tf.ones([4, 5, 5, 5]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 5 ,7]))
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,2,2,1], use_cudnn_on_gpu=False, padding='SAME')
oplist.append([op2, "case 8"])

#############################################################

input_arg  = tf.Variable(tf.ones([1, 4, 4, 1]))
filter_arg = tf.Variable(tf.ones([3 ,3 , 1 ,1]))

op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,1,1,1], use_cudnn_on_gpu=False, padding='VALID')
oplist.append([op2, "case 21"])

#############################################################


#equals to fully padding conv2d!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#diffence is output_shape  replace padding?U can't use padding='SAME',just'valid'
# [batch, in_height, in_width, in_channels]
input_arg  = tf.Variable(tf.ones([1, 2, 2, 1])*9)
# [filter_height, filter_width, in_channels, out_channels]
filter_arg = tf.Variable(tf.ones([3 ,3 , 1 ,1]))

op2 = tf.nn.conv2d_transpose(input_arg, filter_arg, output_shape = [1,4,4,1], strides=[1,1,1,1], padding='VALID')

oplist.append([op2, "case 22"])

#you have to match   output_shape and strides and padding  manually!!!!!!!!!!!!!!!!!!!
# [batch, in_height, in_width, in_channels]
input_arg  = tf.Variable(tf.ones([1, 2, 2, 1])*9)
# [filter_height, filter_width, in_channels, out_channels]
filter_arg = tf.Variable(tf.ones([3 ,3 , 1 ,1]))

op2 = tf.nn.conv2d_transpose(input_arg, filter_arg, output_shape = [1,5,5,1], strides=[1,2,2,1], padding='VALID')

oplist.append([op2, "case 23"])

#conv2d_transpose not equals to conv2d
# [batch, in_height, in_width, in_channels]
input_arg  = tf.Variable(tf.ones([1, 2, 2, 1])*9)
# [filter_height, filter_width, in_channels, out_channels]
filter_arg = tf.Variable(tf.ones([3 ,3 , 1 ,1]))

op2 = tf.nn.conv2d(input_arg, filter_arg,  strides=[1,1,1,1], padding='SAME')

oplist.append([op2, "case 24"])


#pooling
input_arg  = tf.Variable(tf.ones([1, 3, 3, 1]))
op2 = tf.nn.max_pool(value = input_arg, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
oplist.append([op2, "case 25"])
#ksize do not influence padding and output size
input_arg  = tf.Variable(tf.ones([1, 3, 3, 1]))
op2 = tf.nn.max_pool(value = input_arg, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'SAME')
oplist.append([op2, "case 26"])
#ksize do not influence padding and output size
input_arg  = tf.Variable(tf.ones([1, 3, 3, 1]))
op2 = tf.nn.max_pool(value = input_arg, ksize = [1,16,16,1], strides = [1,2,2,1], padding = 'SAME')
oplist.append([op2, "case 27"])
#############################################################################################
#make up datas
input_arg  = tf.Variable(np.array([1,2,3,4,5,6,7,8,9]).reshape(1,3,3,1))
op2 = tf.nn.max_pool(value = input_arg, ksize = [1,16,16,1], strides = [1,1,1,1], padding = 'SAME')
oplist.append([op2, "case 28"])
op2 = input_arg
oplist.append([op2, "original data"])

op2 = tf.nn.max_pool(value = input_arg, ksize = [1,3,3,1], strides = [1,2,2,1], padding = 'SAME')#VALID:[[[[9]]]]
oplist.append([op2, "case 30"])
op2 = tf.nn.max_pool(value = input_arg, ksize = [1,4,4,1], strides = [1,2,2,1], padding = 'SAME')#VALID:error
oplist.append([op2, "case 31"])



# if strides_size == input_size,SAME has no effect!!!!!!!!!!!!!!
# input_arg  = tf.Variable(tf.ones([1, 3, 3, 1]))
# # [filter_height, filter_width, in_channels, out_channels]
# filter_arg = tf.Variable(tf.ones([2 ,2 , 1 ,1]))
# op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,3,3,1], use_cudnn_on_gpu=False, padding='SAME')
# oplist.append([op2, "case 61"])


# [batch, in_height, in_width, in_channels]
input_arg  = tf.Variable(tf.ones([1, 11, 11, 1]))
# [filter_height, filter_width, in_channels, out_channels]
filter_arg = tf.Variable(tf.ones([3 ,3 , 1 ,1]))
op2 = tf.nn.conv2d(input_arg, filter_arg, strides=[1,2,2,1], use_cudnn_on_gpu=False, padding='SAME')
oplist.append([op2, "case 61"])


#################################################################################################

with tf.Session() as a_sess:
    a_sess.run(tf.global_variables_initializer())
    for aop in oplist:
        print("----------{}---------".format(aop[1]))
        print(a_sess.run(aop[0]))
        print('---------------------\n\n')


----------case 2---------
[[[[ 45.]
   [ 45.]
   [ 45.]]

  [[ 45.]
   [ 45.]
   [ 45.]]

  [[ 45.]
   [ 45.]
   [ 45.]]]]
---------------------


----------case 2---------
[[[[ 5.]
   [ 5.]
   [ 5.]]

  [[ 5.]
   [ 5.]
   [ 5.]]

  [[ 5.]
   [ 5.]
   [ 5.]]]]
---------------------


----------case 3---------
[[[[ 45.]]]]
---------------------


----------case 4---------
[[[[ 45.]
   [ 45.]
   [ 45.]]

  [[ 45.]
   [ 45.]
   [ 45.]]

  [[ 45.]
   [ 45.]
   [ 45.]]]]
---------------------


----------case 5---------
[[[[ 20.]
   [ 30.]
   [ 30.]
   [ 30.]
   [ 20.]]

  [[ 30.]
   [ 45.]
   [ 45.]
   [ 45.]
   [ 30.]]

  [[ 30.]
   [ 45.]
   [ 45.]
   [ 45.]
   [ 30.]]

  [[ 30.]
   [ 45.]
   [ 45.]
   [ 45.]
   [ 30.]]

  [[ 20.]
   [ 30.]
   [ 30.]
   [ 30.]
   [ 20.]]]]
---------------------


----------case 6---------
[[[[ 20.  20.  20.  20.  20.  20.  20.]
   [ 30.  30.  30.  30.  30.  30.  30.]
   [ 30.  30.  30.  30.  30.  30.  30.]
   [ 30.  30.  30.  30.  30.  30.  30.]
   [ 20.